# Franka Robot Grasping Tutorial: With and Without Spinning Pads

This tutorial demonstrates a comparison between two Franka Emika Panda robots—one with spinning pads and one without—performing the same grasping task. The spinning pads reduce the torsional friction about the normal vector of the contact between robot's finger and the object. 
"The implementation for friction in genesis is not as advanced that in Mujoco. There is only 1 scalar parameter in Genesis, and condim cannot be changed (only condim = 3 is implemented). There is no torsional friction at the moment. The torsional friction in genesis is the result of multi-contact friction."


The standard robot uses joints 0-8, with 0-6 for arm movement and 7-8 for the gripper. The modified robot includes spinning pads at joints 9-10, which can help provide rotational slippage at the contact point

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner


import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

[I 05/18/25 22:15:10.044 8225] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
gs.destroy()
gs.init(backend=gs.cpu)
gs.set_random_seed(seed=42)
adapter = GenesisAdapter()
planner = ScrewMotionPlanner()

[Genesis] [22:15:13] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [22:15:13] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [22:15:13] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [22:15:13] [INFO] Running on [11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz] with backend gs.cpu. Device memory: 31.05 GB.
[Genesis] [22:15:13] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [22:15:13] [INFO] Scene <c1d0485> created.


## Scene Construction

Here we create the simulation environment with:
1. A ground plane
2. Two Franka Emika Panda robots:
   - One standard robot at position (0,2,0)
   - One with spinning pads and virtual finger at position (0,0,0)
3. Two cubes for each robot
4. A target entity for visualization

The key difference between the robots is that the standard Franka `(franka_MJCF)` uses only the arm and finger joints (joints 0-8), while the other version has additional spinning pads (joints 9-10) that cause rotational slippage using extra hinge DoFs.


In [3]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)

franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda_vir.xml")
                   
)
cube_size = (0.1, 0.07, 0.25)
cube_pos = (0.7, 0, 0.05)

cube = scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = cube_pos,
        euler = (0, -90, 0)
    ),
    surface=gs.surfaces.Default(
        color=(0.5, 0.8, 0.94),
    ),
    material=gs.materials.Rigid(friction=5)
    
)

bottle = scene.add_entity(
    material=gs.materials.Rigid(rho=300),
    morph=gs.morphs.URDF(
        file="urdf/3763/mobility_vhacd.urdf",
        scale=0.09,
        pos=(0.65, 0.2, 0.036),
        euler = (30, -90, 0),
    ),
    # visualize_contact=True,
)

cylinder_pos = (0.7, -0.2, 0.03)
cylinder_euler = (-30, -90, 0)
cylinder_radius = 0.03
cylinder_height = 0.15

cylinder = scene.add_entity(
    gs.morphs.Cylinder(
        radius = cylinder_radius,
        height = cylinder_height,
        pos    = cylinder_pos,
        euler  = cylinder_euler,
    ),
    surface=gs.surfaces.Default(
        color = (0.1, 0.8 , 0.1),   
    )
)

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

[Genesis] [22:15:13] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <fa2748d>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [22:15:13] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <fe3a808>, morph: <gs.morphs.MJCF(file='/home/vahid/repos/pandaSim/assets/xml/franka_emika_panda/panda_vir.xml')>, material: <gs.materials.Rigid>.
[Genesis] [22:15:14] [WARNING] (MJCF) Tendon not supported
[Genesis] [22:15:14] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [22:15:14] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [22:15:14] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [22:15:14] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [22:15:14] [INFO] Adding <gs.RigidEntity>. idx: 2, 

## Robot Configuration

We build the scene and define important joint groups:
- `motors_dof`: Joints 0-6 for arm movement
- `fingers_dof`: Joints 7-8 for gripper fingers
- `virtual_hand`: Joint 9 at the middle of the finger for rotational slippage (only on the `franka`)
- `spinning_pads`: Joints 9-10 for the spinning contact pads (only on the `franka`)

In [4]:
n_envs = 0
scene.build(compile_kernels=False)


[Genesis] [22:15:16] [INFO] Building scene <c1d0485>...


[W 05/18/25 22:15:16.388 8225] [frontend_ir.cpp:begin_frontend_struct_for_on_snode@1678] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [22:15:16] [WARNING] Reference robot position exceeds joint limits.


[W 05/18/25 22:15:16.659 8225] [frontend_ir.cpp:begin_frontend_struct_for_on_snode@1678] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.
[W 05/18/25 22:15:16.731 8225] [frontend_ir.cpp:begin_frontend_struct_for_on_snode@1678] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.
[W 05/18/25 22:15:17.380 8225] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [22:15:18] [INFO] Building visualizer...


[W 05/18/25 22:15:18.680 8225] [frontend_ir.cpp:begin_frontend_struct_for_on_snode@1678] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [22:15:20] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:
for joint in franka.joints:
    print(joint.name, joint.idx_local)
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)
virtual_finger = 9
spinning_pads = np.arange(10, 12)

joint1 0
joint2 1
joint3 2
joint4 3
joint5 4
joint6 5
joint7 6
finger_joint1 7
finger_joint2 8
virtual_finger_joint 9
left_pad_hinge 10
right_pad_hinge 11


## Initial Robot Positioning

Reset the scene and set both robots to the same initial joint configuration. We'll use this "ready pose" as a starting position for our task. Notice that the standard robot has 9 DOFs (degrees of freedom), while the modified version can have 1 dof extra for virtual_finger and two extras for spinning_pads

In [6]:
n_envs = 0

In [50]:
scene.reset()
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])
virtual_franka_qr = np.append(qr, [0, 0, 0])
ready_qpos = virtual_franka_qr
# set joints position
franka.ready_qpos = ready_qpos.squeeze()
franka.set_qpos(franka.ready_qpos)
# position control
franka.control_dofs_position(franka.ready_qpos)


end_effector = franka.get_link('hand')
virtual_finger = franka.get_link('virtual_finger')



target_entity.set_pos(virtual_finger.get_pos())
target_entity.set_quat(virtual_finger.get_quat())


scene.step()

[Genesis] [22:59:07] [INFO] Resetting Scene <c1d0485>.


# Find Grasp Pose for Cube

In this section, we'll find a suitable grasp pose for the cube. The planner.compute_grasp function allows us to specify different grasp approaches:
- grasp_height='top': Approaches to the top of the object
- grasp_height='bottom': Approaches to the bottom of the object
- grasp_height='center': Approaches to the center of the object

Setting prefer_closer_grasp=True ensures the planner finds a grasp pose that minimizes the distance between the current gripper position and the target grasp position, which is useful for efficiency and to avoid unnecessary movements.

Make sure to include the displacement offset if you'are using 'hand' or another link as end_effector link to grasp


In [51]:
scene.n_envs

0

In [52]:
scene.step()
grasp_T_gripper = np.array([[0, 1, 0, 0],
                          [0, 0, -1, 0],
                          [-1, 0, 0, 0],
                          [0, 0, 0, 1]])

grasp_pose, qs, s_axes = planner.compute_grasp(obj=cube, 
                                          adapter=adapter, 
                                          prefer_closer_grasp=True,
                                          grasp_height='top',
                                          offset_toward=0.02,
                                          gripper_offset=grasp_T_gripper,
                                          output_type='pq')

target_entity.set_qpos(grasp_pose)
scene.step()

## Pre-grasp Positioning

Now we'll move both robots to a position above the cube. We'll use inverse kinematics to calculate the joint positions needed to place the end-effector at the desired position and orientation. 


In [53]:

qpos = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = grasp_pose[:3] + np.array([0, 0, 0.1]),
    quat = grasp_pose[3:],
    pos_tol = 1e-5,
    rot_mask = [False, False, True]
)
qpos[fingers_dof] = 0.04


In [54]:
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 250, # 2s duration
)

[Genesis] [22:59:21] [INFO] Path solution found successfully.


## Path Planning

Generate a smooth trajectory for each robot to reach the pre-grasp position.

In [55]:
# execute the planned path
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
adapter.step_simulation(1)

## Reaching for the Object

Calculate and execute a path to move from the pre-grasp position down to the actual grasp position. Both robots move their end-effectors to the side of their respective cubes.


In [56]:

qpos = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = grasp_pose[:3],
    quat = grasp_pose[3:],
    pos_tol = 1e-5,
    rot_mask = [False, False, True]
)
qpos[fingers_dof] = 0.04


In [57]:
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 250, # 2s duration
)

[Genesis] [22:59:52] [INFO] Path solution found successfully.


In [58]:
# execute the planned path
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
adapter.step_simulation(1)

In [59]:
franka.control_dofs_force(np.array([-2, -2]), fingers_dof)
adapter.step_simulation(1)

In [60]:
trajectory = planner.plan(robot=franka, 
                          link=virtual_finger, 
                          object=cube, 
                          adapter=adapter, 
                          qs=qs,
                          s_axes=s_axes,
                          output_type='pq')

In [61]:

qposs = []
for pq in trajectory:
    qpos = franka.inverse_kinematics(
        link = virtual_finger,
        pos = pq[:3],
        quat = pq[3:],
        init_qpos = qpos,
        pos_tol = 1e-5,
        rot_mask = [False, False, True]
    )
    qposs.append(qpos)

In [62]:

for waypoint in qposs:
    franka.control_dofs_position(waypoint[motors_dof], motors_dof)
    target_entity.set_pos(virtual_finger.get_pos())
    target_entity.set_quat(virtual_finger.get_quat())
    scene.step()



In [63]:
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
adapter.step_simulation(1)

In [66]:
finger_pose = adapter.forward_kinematics(franka, virtual_finger, output_type='pq')
finger_pose

array([ 8.9518577e-01, -2.3268118e-04,  2.1986149e-01,  6.8142742e-01,
        6.8108791e-01, -1.8942007e-01,  1.8946242e-01], dtype=float32)

In [72]:
qpos = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = finger_pose[:3] + np.array([0, 0, 0.1]),
    quat = finger_pose[3:],
    pos_tol = 1e-5,
    rot_mask = [False, False, True]
)
qpos[fingers_dof] = 0.04
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 250, # 2s duration
)
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
adapter.step_simulation(1)



[Genesis] [23:06:08] [INFO] Ingoring collision pairs already active for starting pos.
[Genesis] [23:06:08] [INFO] Path solution found successfully.
